In [3]:
from flask import Flask, render_template
from flask_mysqldb import MySQL
from flask_pymongo import PyMongo

import requests
import json

In [4]:
app = Flask(__name__)

app.config["MYSQL_HOST"] = "localhost"
app.config["MYSQL_USER"] = "root"
app.config["MYSQL_PASSWORD"] = "password"
app.config["MYSQL_DB"] = "testdb"
mysql = MySQL(app)

app.config["MONGO_URI"] = "mongodb://localhost:27017/testdb"
mongo = PyMongo(app)

In [5]:
def sql_data():
    with app.app_context():
        cur = mysql.connection.cursor()
        cur.execute("""SELECT * FROM test""")
        rv = cur.fetchall()
        cur.close()
        return rv

In [7]:
for i in sql_data():
    print(i)

('Inger Kearney', 'argelia.hair780@gmail.com', 'Richmond', 'Benin')
('Mathilda Horn', 'dung_bonilla@knights.com', 'York', 'Jordan')
('Jeromy Herzog', 'zula6980@throws.com', 'Sterling Heights', 'Uruguay')
('Reena Spears', 'neida8227@writers.com', 'Scottsdale', 'Libya')
('Liane Coleman', 'wanetta-laws33577@gmail.com', 'Birmingham', 'Malta')
('Jo Henley', 'salvatorehenley78518@consolidated.com', 'Anaheim', 'Latvia')
('Carter Holcomb', 'marion7@recipe.com', 'Pueblo', 'Panama')
('Addie Singletary', 'latesha9@gmail.com', 'New York', 'Jamaica')
('Ilona Mcgrew', 'simona_prince970@gmail.com', 'New York', 'Belize')
('Jovan Apodaca', 'coreen1115@yahoo.com', 'Yonkers', 'Benin')


In [8]:
def mongo_data():
    data = mongo.db.test.find()
    return data

In [9]:
for i in mongo_data():
    print(i)

{'_id': 8752, 'title': 'Divine Comedy', 'author': 'Dante', 'copies': 1}
{'_id': 7000, 'title': 'The Odyssey', 'author': 'Homer', 'copies': 10}
{'_id': 7020, 'title': 'Iliad', 'author': 'Homer', 'copies': 10}
{'_id': 8645, 'title': 'Eclogues', 'author': 'Dante', 'copies': 2}
{'_id': 8751, 'title': 'The Banquet', 'author': 'Dante', 'copies': 2}


In [10]:
@app.route("/")
def index():
    return render_template('index.html')

In [11]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [21/Apr/2023 22:12:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2023 22:12:25] "GET /favicon.ico HTTP/1.1" 404 -
